In [19]:
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Row, SQLContext}
//import org.apache.spark.ml.clustering.{KMeans, KMeansModel, GaussianMixture, GaussianMixtureModel, BisectingKMeans, BisectingKMeansModel}
import org.apache.spark.ml.clustering._
import org.apache.spark.ml.evaluation.ClusteringEvaluator
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator,BinaryClassificationEvaluator}
import org.apache.spark.ml.feature.{StandardScaler, VectorAssembler, StringIndexer, MinMaxScaler}
import org.apache.spark.ml.{Pipeline}
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import java.io.{File, PrintWriter}
import java.text.SimpleDateFormat
import java.util.Calendar

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Row, SQLContext}
import org.apache.spark.ml.clustering._
import org.apache.spark.ml.evaluation.ClusteringEvaluator
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator, BinaryClassificationEvaluator}
import org.apache.spark.ml.feature.{StandardScaler, VectorAssembler, StringIndexer, MinMaxScaler}
import org.apache.spark.ml.Pipeline
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import java.io.{File, PrintWriter}
import java.text.SimpleDateFormat
import java.util.Calendar


In [20]:
val raw = spark.read.format("csv").option("header", "true").option("mode", "DROPMALFORMED").csv("datasets/creditcard.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

raw: org.apache.spark.sql.DataFrame = [Time: string, V1: string ... 29 more fields]


In [21]:
// cast all the column to Double type.
val df = raw.select(((1 to 28).map(i => "V" + i) ++ Array("Time", "Amount", "Class")).map(s => col(s).cast("Double")): _*)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 29 more fields]


In [22]:
// convert the label from {0, 1} to {1, 2}
//val labelConverter = new FuncTransformer(udf {d: Double => if (d==0) 2 else d }).setInputCol("Class").setOutputCol("Class")
val labelConverter = new StringIndexer().setInputCol("Class").setOutputCol("label")
val assembler = new VectorAssembler().setInputCols(Array("V3", "V4", "V9", "V10", "V11", "V12", "V14", "V16", "V17", "V18","V19")).setOutputCol("assembled")
//val scaler = new MinMaxScaler().setInputCol("assembled").setOutputCol("features")
val scaler = new StandardScaler().setInputCol("assembled").setOutputCol("features")
val pipeline = new Pipeline().setStages(Array(assembler, scaler, labelConverter))
val pipelineModel = pipeline.fit(df)
val data = pipelineModel.transform(df)
println("Generate feature from raw data:")
data.select("features", "label").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

labelConverter: org.apache.spark.ml.feature.StringIndexer = strIdx_25c67eefc1bf
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_2d989a639bb4
scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_6fc4ae5344cd
pipeline: org.apache.spark.ml.Pipeline = pipeline_b9e3a65e67f1
pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_b9e3a65e67f1
data: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 32 more fields]
Generate feature from raw data:
+--------------------+-----+
|            features|label|
+--------------------+-----+
|[1.67277056254252...|  0.0|
|[0.10979690934882...|  0.0|
|[1.16946643973207...|  0.0|
|[1.18251437486170...|  0.0|
|[1.02140988238942...|  0.0|
|[0.75258405639254...|  0.0|
|[0.02992291760621...|  0.0|
|[0.70857499080745...|  0.0|
|[-0.0746524907375...|  0.0|
|[0.68878028300438...|  0.0|
|[0.60270853498396...|  0.0|
|[-0.5766178509613...|  0.0|
|[0.25320948651198...|  0.0|
|[0.54648639167204...|  0.0|
|[1.08276652341442.

In [23]:
//Splitting dataset
val splitTime = data.stat.approxQuantile("Time", Array(0.7), 0.001).head
    val trainingData = data.filter(s"Time<$splitTime").cache()
    val validationData = data.filter(s"Time>=$splitTime").cache()
    println("Split data into Training and Validation: ")
    println("training records count: " + trainingData.count())
    println("validation records count: " + validationData.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

splitTime: Double = 132831.0
trainingData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
Split data into Training and Validation: 
training records count: 199155
validation records count: 85652


### 4)-Training the models

In [24]:
// Create a Kmeans Model with K=2
// train the model
val kmeans = new KMeans().setK(2).setFeaturesCol("features").setPredictionCol("clusters").setSeed(1L).setMaxIter(100)
val bisectingkmeans = new BisectingKMeans().setK(2).setFeaturesCol("features").setPredictionCol("clusters").setSeed(1L).setMaxIter(100)
val gaussianMixture = new GaussianMixture().setK(2).setFeaturesCol("features").setPredictionCol("clusters").setSeed(1L).setMaxIter(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

kmeans: org.apache.spark.ml.clustering.KMeans = kmeans_38f606ec88d3
bisectingkmeans: org.apache.spark.ml.clustering.BisectingKMeans = bisecting-kmeans_14b4abc832ce
gaussianMixture: org.apache.spark.ml.clustering.GaussianMixture = GaussianMixture_6d798f46669a


In [25]:
// Fit that model to the training_data
val t = System.nanoTime
val modelKmeans = kmeans.fit(trainingData)
val durationkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationkmeans secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 545839927158490
modelKmeans: org.apache.spark.ml.clustering.KMeansModel = kmeans_38f606ec88d3
durationkmeans: Double = 4.842104795

initial model training finished.
Training process takes 4.842104795 secs


In [26]:
// Fit that model to the training_data
val t = System.nanoTime
val modelBisectingkmeans = bisectingkmeans.fit(trainingData)
val durationbisectingkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationbisectingkmeans secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 545847249076156
modelBisectingkmeans: org.apache.spark.ml.clustering.BisectingKMeansModel = bisecting-kmeans_14b4abc832ce
durationbisectingkmeans: Double = 27.809898236

initial model training finished.
Training process takes 27.809898236 secs


In [27]:
// Fit that model to the training_data
val t = System.nanoTime
val modelGMM = gaussianMixture.fit(trainingData)
val durationGMM = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationGMM secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 545876986190815
modelGMM: org.apache.spark.ml.clustering.GaussianMixtureModel = GaussianMixture_6d798f46669a
durationGMM: Double = 9.410226924

initial model training finished.
Training process takes 9.410226924 secs


### 5)- Prediction

In [28]:
 // Make predictions Kmeans
val t = System.nanoTime
var predictionsk = modelKmeans.transform(validationData)
val durationkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationkmeans secs")
predictionsk = predictionsk.select(col("label"),col("clusters").cast("Double"),col("features"))
//val df = raw.select(((1 to 28).map(i => "V" + i) ++ Array("Time", "Amount", "Class")).map(s => col(s).cast("Double")): _*)
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsk.groupBy("clusters").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 545888353096704
predictionsk: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 33 more fields]
durationkmeans: Double = 0.275742023

initial model training finished.
Training process takes 0.275742023 secs
predictionsk: org.apache.spark.sql.DataFrame = [label: double, clusters: double ... 1 more field]
Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85544|
|  1.0|  108|
+-----+-----+

Prediction :
+--------+-----+
|clusters|count|
+--------+-----+
|     0.0|37723|
|     1.0|47929|
+--------+-----+



In [29]:
predictionsk.select("clusters", "label").groupBy("clusters", "label").count().orderBy("label", "clusters").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+-----+
|clusters|label|count|
+--------+-----+-----+
|     0.0|  0.0|37664|
|     1.0|  0.0|47880|
|     0.0|  1.0|   59|
|     1.0|  1.0|   49|
+--------+-----+-----+



In [30]:
 // Make predictions Bisecting Kmeans 
val t = System.nanoTime
var predictionsbk = modelBisectingkmeans.transform(validationData)
val durationkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationkmeans secs")
predictionsbk = predictionsbk.select(col("label"),col("clusters").cast("Double"),col("features"))
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsbk.groupBy("clusters").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 545892353507186
predictionsbk: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 33 more fields]
durationkmeans: Double = 0.272236378

initial model training finished.
Training process takes 0.272236378 secs
predictionsbk: org.apache.spark.sql.DataFrame = [label: double, clusters: double ... 1 more field]
Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85544|
|  1.0|  108|
+-----+-----+

Prediction :
+--------+-----+
|clusters|count|
+--------+-----+
|     0.0|48498|
|     1.0|37154|
+--------+-----+



In [31]:
predictionsbk.select("clusters", "label").groupBy("clusters", "label").count().orderBy("label", "clusters").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+-----+
|clusters|label|count|
+--------+-----+-----+
|     0.0|  0.0|48446|
|     1.0|  0.0|37098|
|     0.0|  1.0|   52|
|     1.0|  1.0|   56|
+--------+-----+-----+



In [32]:
// Make predictions Bisecting Kmeans 
val t = System.nanoTime
var predictionsgmm = modelGMM.transform(validationData)
val durationkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationkmeans secs")
predictionsgmm = predictionsgmm.select(col("label"),col("clusters").cast("Double"),col("features"))
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsgmm.groupBy("clusters").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 545896889141948
predictionsgmm: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 34 more fields]
durationkmeans: Double = 0.280427711

initial model training finished.
Training process takes 0.280427711 secs
predictionsgmm: org.apache.spark.sql.DataFrame = [label: double, clusters: double ... 1 more field]
Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85544|
|  1.0|  108|
+-----+-----+

Prediction :
+--------+-----+
|clusters|count|
+--------+-----+
|     0.0|61381|
|     1.0|24271|
+--------+-----+



In [33]:
predictionsgmm.select("clusters", "label").groupBy("clusters", "label").count().orderBy("label", "clusters").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+-----+
|clusters|label|count|
+--------+-----+-----+
|     0.0|  0.0|61306|
|     1.0|  0.0|24238|
|     0.0|  1.0|   75|
|     1.0|  1.0|   33|
+--------+-----+-----+



### 6)-Evaluation

In [34]:
val evaluatork1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("accuracy")
val evaluatork2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedPrecision")
val evaluatork3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedRecall")
val evaluatork4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("f1")
val areaUnderROCk = new BinaryClassificationEvaluator().setRawPredictionCol("clusters").setLabelCol("label").setMetricName("areaUnderROC")
val accuracyk = evaluatork1.evaluate(predictionsk)
println("Area Under ROC Curve = " + areaUnderROCk.evaluate(predictionsk))
println("Accuracy = " + evaluatork1.evaluate(predictionsk))
println("Precision = " + evaluatork2.evaluate(predictionsk))
println("Recall = " + evaluatork3.evaluate(predictionsk))
println("F1 = " + evaluatork4.evaluate(predictionsk))
println("Test Error = " + (1.0 - accuracyk))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluatork1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_64913631471c
evaluatork2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_572879f9ced9
evaluatork3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_8f2a13d25235
evaluatork4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_f0033ef6bc30
areaUnderROCk: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_f5afe190c8d4
accuracyk: Double = 0.44030495493391864
Area Under ROC Curve = 0.4469958713038298
Accuracy = 0.44030495493391864
Precision = 0.997178312387319
Recall = 0.4403049549339187
F1 = 0.6103282694480973
Test Error = 0.5596950450660814


In [35]:
val evaluatorbk1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("accuracy")
val evaluatorbk2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedPrecision")
val evaluatorbk3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedRecall")
val evaluatorbk4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("f1")
val areaUnderROCbk = new BinaryClassificationEvaluator().setRawPredictionCol("clusters").setLabelCol("label").setMetricName("areaUnderROC")
val accuracybk = evaluatorbk1.evaluate(predictionsbk)
println("Area Under ROC Curve = " + areaUnderROCbk.evaluate(predictionsbk))
println("Accuracy = " + evaluatorbk1.evaluate(predictionsbk))
println("Precision = " + evaluatorbk2.evaluate(predictionsbk))
println("Recall = " + evaluatorbk3.evaluate(predictionsbk))
println("F1 = " + evaluatorbk4.evaluate(predictionsbk))
println("Test Error = " + (1.0 - accuracybk))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluatorbk1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_a3c05ea4a516
evaluatorbk2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_26b46dcdcc18
evaluatorbk3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_dd8cc7abf847
evaluatorbk4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_0aa78454c9e0
areaUnderROCbk: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_37aebc8a7cfc
accuracybk: Double = 0.5662681548591977
Area Under ROC Curve = 0.542423478842164
Accuracy = 0.5662681548591977
Precision = 0.9976701270404935
Recall = 0.5662681548591977
F1 = 0.7219404016581634
Test Error = 0.4337318451408023


In [36]:
val evaluatorgmm1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("accuracy")
val evaluatorgmm2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedPrecision")
val evaluatorgmm3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedRecall")
val evaluatorgmm4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("f1")
val areaUnderROCgmm = new BinaryClassificationEvaluator().setRawPredictionCol("clusters").setLabelCol("label").setMetricName("areaUnderROC")
val accuracygmm = evaluatorgmm1.evaluate(predictionsgmm)
println("Area Under ROC Curve = " + areaUnderROCgmm.evaluate(predictionsgmm))
println("Accuracy = " + evaluatorgmm1.evaluate(predictionsgmm))
println("Precision = " + evaluatorgmm2.evaluate(predictionsgmm))
println("Recall = " + evaluatorgmm3.evaluate(predictionsgmm))
println("F1 = " + evaluatorgmm4.evaluate(predictionsgmm))
println("Test Error = " + (1.0 - accuracygmm))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluatorgmm1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_1b1f2648b35d
evaluatorgmm2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_355ae279caff
evaluatorgmm3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_e174d3b8f0ad
evaluatorgmm4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_5ad2c85f083d
areaUnderROCgmm: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_e037b2dd8c96
accuracygmm: Double = 0.7161420632326158
Area Under ROC Curve = 0.5111079938069557
Accuracy = 0.7161420632326158
Precision = 0.9975204623430861
Recall = 0.7161420632326158
F1 = 0.8334721666150667
Test Error = 0.28385793676738424
